# Deploymnet

In [6]:
import aiohttp
import asyncio
import uvicorn
from fastai import *
from fastai.vision import *
from io import BytesIO
from starlette.applications import Starlette
from starlette.middleware.cors import CORSMiddleware
from starlette.responses import HTMLResponse, JSONResponse
from starlette.staticfiles import StaticFiles

In [7]:
export_file_url = 'https://drive.google.com/file/d/1JyjIpXsvWTWY2mvJz5LDNJ6UCgFp5-ME/view?usp=sharing'
export_file_name = 'bbc-classifier.pkl'

In [8]:
async def download_file(url, dest):
    if dest.exists(): return
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.read()
            with open(dest, 'wb') as f:
                f.write(data)


In [12]:
async def setup_learner():
    await download_file(export_file_url, path / export_file_name)
    try:
        learn = load_learner(path, export_file_name)
        return learn
    except RuntimeError as e:
        if len(e.args) > 0 and 'CPU-only machine' in e.args[0]:
            print(e)
            message = "\n\nThis model was trained with an old version of fastai and will not work in a CPU environment.\n\nPlease update the fastai library in your training environment and export your model again.\n\nSee instructions for 'Returning to work' at https://course.fast.ai."
            raise RuntimeError(message)
        else:
            raise


In [10]:
path = Path('/tmp')

In [14]:
download_file(export_file_url, path / export_file_name)

<coroutine object download_file at 0x7fa3b5bb5d40>

In [18]:
export_file_name

'bbc-classifier.pkl'

In [19]:
learn = load_learner(path, export_file_name)

UnpicklingError: invalid load key, '<'.

In [16]:
learn=setup_learner()

In [ ]:
learn.

In [13]:
loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(setup_learner())]
learn = loop.run_until_complete(asyncio.gather(*tasks))[0]
loop.close()

RuntimeError: This event loop is already running